# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [69]:
# import libraries
from sqlalchemy import create_engine

import re
import nltk
nltk.download(['stopwords', 'punkt', 'wordnet'])
import pickle
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brunoyang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/brunoyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brunoyang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
# read in file
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster', engine)
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [53]:
def tokenize(text):
    # Define common paras
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # Normalize and remove punctuation
    text = re.sub(r'[^a-zA-Z0-9]', '', text.lower())
    
    # Tokenize text
    text = word_tokenize(text)
    
    # Lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]
    
    # return tokens
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [54]:
pipeline = Pipeline([
         ('vect', CountVectorizer(tokenizer=tokenize)),
         ('tfidf', TfidfTransformer()),
         ('mutclf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [55]:
# Split train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [56]:
# Predict use the trained model
y_pred = pipeline.predict(X_test)

In [57]:
# Report Model Effectiveness
for i, col in enumerate(y_test.columns):
    target_names = ['class 0', 'class 1', 'class 2']
    print(classification_report(y_test[col].tolist(), list(y_pred[:, i]), target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.62      0.01      0.01      1556
    class 1       0.69      0.00      0.00      4952
    class 2       0.01      0.98      0.01        46

avg / total       0.67      0.01      0.01      6554

             precision    recall  f1-score   support

    class 0       0.83      1.00      0.91      5434
    class 1       0.75      0.00      0.01      1120

avg / total       0.82      0.83      0.75      6554

             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00      6531
    class 1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554

             precision    recall  f1-score   support

    class 0       0.59      1.00      0.74      3842
    class 1       0.50      0.00      0.00      2712

avg / total       0.55      0.59      0.43      6554

             precision    recall  f1-score   support

    class 0       0.93      1.00 

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 3
  .format(len(labels), len(target_names))
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 3
  .format(len(labels), len(target_names))


### 6. Improve your model
Use grid search to find better parameters. 

In [62]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False)}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [63]:
# Report New Model Effectiveness
for i, col in enumerate(y_test.columns):
    target_names = ['class 0', 'class 1', 'class 2']
    print(classification_report(y_test[col].tolist(), list(y_pred[:, i]), target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.57      0.01      0.01      1556
    class 1       0.76      1.00      0.86      4952
    class 2       0.00      0.00      0.00        46

avg / total       0.71      0.76      0.65      6554

             precision    recall  f1-score   support

    class 0       0.83      1.00      0.91      5434
    class 1       1.00      0.00      0.00      1120

avg / total       0.86      0.83      0.75      6554

             precision    recall  f1-score   support

    class 0       1.00      1.00      1.00      6531
    class 1       0.00      0.00      0.00        23

avg / total       0.99      1.00      0.99      6554

             precision    recall  f1-score   support

    class 0       0.59      1.00      0.74      3842
    class 1       0.75      0.00      0.00      2712

avg / total       0.65      0.59      0.43      6554

             precision    recall  f1-score   support

    class 0       0.93      1.00 

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 3
  .format(len(labels), len(target_names))
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 3
  .format(len(labels), len(target_names))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [70]:
pipeline2 = Pipeline([
         ('vect', CountVectorizer(tokenizer=tokenize)),
         ('tfidf', TfidfTransformer()),
         ('mutclf', MultiOutputClassifier(AdaBoostClassifier(), n_jobs=-1))])

### 9. Export your model as a pickle file

In [71]:
# Save CV Model
with open('model.pickle', 'wb') as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.